## Installing Necessary Libraries

In [1]:
import torch

print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.1.1+cu121
Is CUDA enabled? True


In [1]:
# !pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
# !pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

# Dataset details
Instacart Data can be downloaded from [here](https://www.kaggle.com/competitions/instacart-market-basket-analysis/data). We just need product & department csv files

In [9]:
import pandas as pd
df_product = pd.read_csv("/code/llm/lmm_vqa/Fine-tuning/llm_finetuning/PEFT/products.csv")
df_dept = pd.read_csv('/code/llm/lmm_vqa/Fine-tuning/llm_finetuning/PEFT/departments.csv')

In [10]:
df_joined = pd.merge(df_product, df_dept, on = ['department_id'])
# Create a 'text' column g=that maps the product name with department
df_joined['text'] = df_joined.apply(lambda row: row['product_name'] + " ->: " + row['department'], axis = 1)

In [11]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_joined, test_size=0.2, random_state=42)

In [12]:
train_df.head(10)

,product_id,product_name,aisle_id,department_id,department,text
7361,10376,Organic Caraway Seeds,104,13,pantry,Organic Caraway Seeds ->: pantry
32534,40131,Original Laundry Detergent,75,17,household,Original Laundry Detergent ->: household
1457,11913,Shelled Pistachios,117,19,snacks,Shelled Pistachios ->: snacks
5201,41392,Harvest Berry Chewy Granola Bars,3,19,snacks,Harvest Berry Chewy Granola Bars ->: snacks
38539,28627,Veganic Sprouted Red Fife Raisin Bran,121,14,breakfast,Veganic Sprouted Red Fife Raisin Bran ->: brea...
18377,29407,Root Vegetable Cakes,42,1,frozen,Root Vegetable Cakes ->: frozen
28627,29571,Alta Dena 1% Milk,84,16,dairy eggs,Alta Dena 1% Milk ->: dairy eggs
30283,3589,Laundry Detergent Pods,75,17,household,Laundry Detergent Pods ->: household
32030,31685,"Daily Shower Cleaner Scrub Free Refill, Fresh ...",114,17,household,"Daily Shower Cleaner Scrub Free Refill, Fresh ..."
28655,30119,Organic French Style Meyer Lemon Yogurt,120,16,dairy eggs,Organic French Style Meyer Lemon Yogurt ->: da...


In [13]:
test_df.head(10)

,product_id,product_name,aisle_id,department_id,department,text
33626,24154,Free & Clear Stage 4 Overnight Diapers,56,18,babies,Free & Clear Stage 4 Overnight Diapers ->: babies
18192,27348,"Beef pot roast with roasted potatoes, carrots,...",38,1,frozen,"Beef pot roast with roasted potatoes, carrots,..."
47099,27181,Coffee Liquer,124,5,alcohol,Coffee Liquer ->: alcohol
48183,20577,Bread Rolls,43,3,bakery,Bread Rolls ->: bakery
22197,16472,French Milled Oval Almond Gourmande Soap,25,11,personal care,French Milled Oval Almond Gourmande Soap ->: p...
31573,24121,Dust Pan,114,17,household,Dust Pan ->: household
45362,5477,Roasted Pine Nut Hommus,67,20,deli,Roasted Pine Nut Hommus ->: deli
14131,27921,Cranberry Raspberry Juice Cocktail,98,7,beverages,Cranberry Raspberry Juice Cocktail ->: beverages
26903,4786,Sweet Cream Butter Salted,36,16,dairy eggs,Sweet Cream Butter Salted ->: dairy eggs
39417,8796,Traditional Chicken Barley Soup,69,15,canned goods,Traditional Chicken Barley Soup ->: canned goods


In [14]:
from datasets import Dataset,DatasetDict

train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df),
})

## Loading the model

In [18]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="cuda:0"
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

[codecarbon INFO @ 19:32:14] Energy consumed for RAM : 0.001561 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:32:14] Energy consumed for all GPUs : 0.006024 kWh. Total GPU Power : 84.09700000000001 W
[codecarbon INFO @ 19:32:14] Energy consumed for all CPUs : 0.001417 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:32:14] 0.009002 kWh of electricity used since the beginning.


[codecarbon INFO @ 19:32:29] Energy consumed for RAM : 0.001756 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:32:29] Energy consumed for all GPUs : 0.006372 kWh. Total GPU Power : 83.376 W
[codecarbon INFO @ 19:32:29] Energy consumed for all CPUs : 0.001594 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:32:29] 0.009721 kWh of electricity used since the beginning.


Let's also load the tokenizer below

In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [20]:
tokenizer.pad_token = tokenizer.eos_token

[codecarbon INFO @ 19:32:44] Energy consumed for RAM : 0.001951 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:32:44] Energy consumed for all GPUs : 0.006577 kWh. Total GPU Power : 49.245000000000005 W
[codecarbon INFO @ 19:32:44] Energy consumed for all CPUs : 0.001771 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:32:44] 0.010299 kWh of electricity used since the beginning.


**Let's check what the base model predicts before finetuning. :)**

In [5]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="cuda:0",
)

# Try to generate text for some example produce names
sequences = pipeline(
   ["“Free & Clear Stage 4 Overnight Diapers” ->:","Bread Rolls ->:","French Milled Oval Almond Gourmande Soap ->:"],
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq[0]['generated_text']}")

/home/yushi/anaconda3/envs/autotrain/lib/python3.11/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Result: “Free & Clear Stage 4 Overnight Diapers” ->: $0. paździera 2014 16:31:25
194 “Free & Clear Stage 1 Baby Wipes” ->: $0. paździera 2014 16:30:58
193 “Free & Clear Stage 1 Baby Wipes” ->: $0. wrzesień 2014 21:01:08
192 “Free & Clear Stage 1 Baby Wipes” ->: $0. wrzesień 2014 21:01:03
191 “Free & Clear Stage 2 Diapers” ->: $0. wrzesień 2014 21:00:20
190 “Free & Clear Stage 2 Diapers” ->: $
Result: Bread Rolls ->:
 kwietnika-gwitkowie-garnitury-wszystkie-w-garniturach-na-wilgotna-zima-2019/
https://www.wilkosupermarket.pl/produkt/bakalarz-kobieca-z-gwiazdkami-czerwone-265145225/
https://www.wilkopas.pl/produkt/koszulka-bajka-lila/
http://baza-danych.papieros-sportowy.pl/2018/04/23/golfowe-koszulki-mensualne-mens-2015-265124346/
https://www.wilkopas
Result: French Milled Oval Almond Gourmande Soap ->: $14. sierp 2019, 12:21.
Floral and herbal. This is the best soap I have ever used and I have used a lot! It is a very mild, creamy and delicately perfumed soap. This is my favorite soap 

Below we will load the configuration file in order to create the LoRA model. According to QLoRA paper, it is important to consider all linear layers in the transformer block for maximum performance.

In [6]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","v_proj"]
)

## Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

In [21]:
from transformers import TrainingArguments

output_dir = "./llama2_instacard_results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 120
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

Then finally pass everthing to the trainer

In [22]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_dict['train'],
    # train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    # dataset_text_field="prediction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/39750 [00:00<?, ? examples/s]

[codecarbon INFO @ 19:32:59] Energy consumed for RAM : 0.002146 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:32:59] Energy consumed for all GPUs : 0.006920 kWh. Total GPU Power : 82.396 W
[codecarbon INFO @ 19:32:59] Energy consumed for all CPUs : 0.001948 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:32:59] 0.011014 kWh of electricity used since the beginning.
/home/yushi/anaconda3/envs/autotrain/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:214: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
[codecarbon INFO @ 19:32:59] [setup] RAM Tracking...
[codecarbon INFO @ 19:32:59] [setup] GPU Tracking...
[codecarbon INFO @ 19:32:59] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 19:32:59] [setup] CPU Tracking...
[c

We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [23]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [24]:
trainer.train()

You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/yushi/anaconda3/envs/autotrain/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1,3.922600
2,4.079400
3,4.115400
4,4.055500
5,4.060000
6,3.984200
7,3.757000
8,3.610600
9,3.644100
10,3.536000


[codecarbon INFO @ 19:33:14] Energy consumed for RAM : 0.002341 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:33:14] Energy consumed for all GPUs : 0.008421 kWh. Total GPU Power : 360.583 W
[codecarbon INFO @ 19:33:14] Energy consumed for all CPUs : 0.002125 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:33:14] 0.012888 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:33:22] Energy consumed for RAM : 0.000195 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:33:22] Energy consumed for all GPUs : 0.001501 kWh. Total GPU Power : 360.104 W
[codecarbon INFO @ 19:33:22] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:33:22] 0.001873 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:33:29] Energy consumed for RAM : 0.002537 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:33:29] Energy consumed for all GPUs : 0.009245 kWh. Total GPU Power : 196.802 W
[codecarbon INFO @ 19:33:29

TrainOutput(global_step=120, training_loss=2.285853257775307, metrics={'train_runtime': 421.5443, 'train_samples_per_second': 9.109, 'train_steps_per_second': 0.285, 'total_flos': 2453200866115584.0, 'train_loss': 2.285853257775307, 'epoch': 0.1})

[codecarbon INFO @ 19:40:14] Energy consumed for RAM : 0.007803 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:40:14] Energy consumed for all GPUs : 0.039471 kWh. Total GPU Power : 134.359 W
[codecarbon INFO @ 19:40:14] Energy consumed for all CPUs : 0.007084 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:40:14] 0.054358 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:40:29] Energy consumed for RAM : 0.007999 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:40:29] Energy consumed for all GPUs : 0.039733 kWh. Total GPU Power : 62.997 W
[codecarbon INFO @ 19:40:29] Energy consumed for all CPUs : 0.007261 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:40:29] 0.054993 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:40:44] Energy consumed for RAM : 0.008194 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:40:44] Energy consumed for all GPUs : 0.039967 kWh. Total GPU Power : 56.284 W
[codecarbon INFO @ 19:40:44] 

#### Inference on a test sample (25 data points)

In [25]:
lst_test_data = list(test_df['text'])

In [26]:
len(lst_test_data)

9938

In [27]:
# Take a test sample of size 25 for inference
sample_size = 25
lst_test_data_short = lst_test_data[:sample_size]

In [28]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="cuda:0",
)

sequences = pipeline(
    lst_test_data_short, # Inference on the small sample test set
    max_length=100,  #200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

# For loop to generate answers for each data point
for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])

0 Free & Clear Stage 4 Overnight Diapers ->: babies: diapers & accessories diapers ->: baby needs diapers & pads free clear diapers ->: baby needs diapers free & clear stage 4 overnight diapers: diapers ->: baby needs diapers diapers & accessories ->: baby needs diapers free & clear stage 4 overnight diapers: diapers ->: personal care baby care ->: diapers & accessories dia
1 Beef pot roast with roasted potatoes, carrots, sweet onions, green beans, and a rich gravy Beef Pot Roast ->: frozen goods ->: frozen meals ->: meat, chicken & seafood ->: beef & beef products ->: entree ->: entree ->: entree ->: entrees: beef ->: beef entree ->: meat ->: beef ->: meat, chicken
2 Coffee Liquer ->: alcoholic beverages spirits ->: beverages beer & hard cider ->: beer coffee liqueur ->: beverages liquor ->: beverages beer & hard cider coffee liquor ->: beverages beer coffee liqueur original ->: beverages liquor ->: beverages beer & hard cider coffee liqueur original alcoholic beverages spirits drinks

[codecarbon INFO @ 19:49:44] Energy consumed for RAM : 0.015217 kWh. RAM Power : 46.85093021392822 W


/home/yushi/anaconda3/envs/autotrain/lib/python3.11/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
[codecarbon INFO @ 19:49:59] Energy consumed for RAM : 0.015413 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:49:59] Energy consumed for all GPUs : 0.048501 kWh. Total GPU Power : 320.163 W
[codecarbon INFO @ 19:49:59] Energy consumed for all CPUs : 0.013990 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:49:59] 0.077904 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:50:14] Energy consumed for RAM : 0.015608 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:50:14] Energy con

0 Free & Clear Stage 4 Overnight Diapers ->: babies kids baby care personal care diapers: personal care products: babies kids babies & kids personal care: babies kids baby care personal care diapers babies & kids personal care babies & kids personal care diapers: babies & kids personal care babies & kids personal care diapers: babies & kids personal care
1 Beef pot roast with roasted potatoes, carrots, sweet onions, green beans, and a rich gravy Beef Pot Roast ->: frozen foods -> frozen meals seafood entrees chicken/beef entrees vegetable entrees meat & poultry -> meats -> beef -> beef pot roast ->: frozen goods frozen beef pot roast frozen meal kit beef pot roast
2 Coffee Liquer ->: alcohol spirits liqueurs & wine wine & beer ->: alcohol spirits liqueurs & wine wine & beer lambic ->: alcohol spirits liqueurs & wine wine & beer lambic ->: alcohol spirits liqueurs & wine wine & beer lambic ->: alcohol spirits liqueurs & wine wine & beer lambic ->: alcohol spirits liqueurs & wine wine & 

[codecarbon INFO @ 19:53:59] Energy consumed for RAM : 0.018534 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:53:59] Energy consumed for all GPUs : 0.069037 kWh. Total GPU Power : 101.7 W
[codecarbon INFO @ 19:53:59] Energy consumed for all CPUs : 0.016824 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:53:59] 0.104395 kWh of electricity used since the beginning.


In [30]:
# Get the model answers - the first arrow answer
def correct_answer(ans):
  return (ans.split("->")[1]).strip()

answers = []
for ix,seq in enumerate(sequences):
    # print(ix,seq[0]['generated_text'])
    answers.append(correct_answer(seq[0]['generated_text']))

answers

[': babies kids baby care personal care diapers: personal care products: babies kids babies & kids personal care: babies kids baby care personal care diapers babies & kids personal care babies & kids personal care diapers: babies & kids personal care babies & kids personal care diapers: babies & kids personal care',
 ': frozen foods',
 ': alcohol spirits liqueurs & wine wine & beer',
 ': bakery products pasta breads pastanella bakery',
 ': personal care personal care bars & bars',
 ': household supplies',
 ': deli produce dairy eggs seafood meat produce beverages alcoholic beverages pantry household personal care babies & kids toys sports outdoors office & school art & crafts auto hobby books & magazines musical instruments, equipment & accessories home improvement hardware appliances electronics shoes, clothing & accessories baby items pet products health care & wellbeing',
 ': beverages alcoholic beverages wine & spirits liquor vodka & grain alcohol vodka & grain alcohol brands cucum

[codecarbon INFO @ 19:56:14] Energy consumed for RAM : 0.020290 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:56:14] Energy consumed for all GPUs : 0.070971 kWh. Total GPU Power : 48.923 W
[codecarbon INFO @ 19:56:14] Energy consumed for all CPUs : 0.018417 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:56:14] 0.109678 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:56:29] Energy consumed for RAM : 0.020485 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:56:29] Energy consumed for all GPUs : 0.071175 kWh. Total GPU Power : 48.891 W
[codecarbon INFO @ 19:56:29] Energy consumed for all CPUs : 0.018594 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:56:29] 0.110254 kWh of electricity used since the beginning.


In [31]:
df_evaluate = test_df.iloc[:sample_size][['product_name','department']]

df_evaluate = df_evaluate.reset_index(drop=True)

df_evaluate['department_predicted'] = answers

df_evaluate

,product_name,department,department_predicted
0,Free & Clear Stage 4 Overnight Diapers,babies,: babies kids baby care personal care diapers:...
1,"Beef pot roast with roasted potatoes, carrots,...",frozen,: frozen foods
2,Coffee Liquer,alcohol,: alcohol spirits liqueurs & wine wine & beer
3,Bread Rolls,bakery,: bakery products pasta breads pastanella bakery
4,French Milled Oval Almond Gourmande Soap,personal care,: personal care personal care bars & bars
5,Dust Pan,household,: household supplies
6,Roasted Pine Nut Hommus,deli,: deli produce dairy eggs seafood meat produce...
7,Cranberry Raspberry Juice Cocktail,beverages,: beverages alcoholic beverages wine & spirits...
8,Sweet Cream Butter Salted,dairy eggs,: dairy eggs eggs cheese dairy eggs cheese sna...
9,Traditional Chicken Barley Soup,canned goods,: canned goods pasta dry goods produce deli me...


[codecarbon INFO @ 19:56:44] Energy consumed for RAM : 0.020680 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:56:44] Energy consumed for all GPUs : 0.071377 kWh. Total GPU Power : 48.707 W
[codecarbon INFO @ 19:56:44] Energy consumed for all CPUs : 0.018771 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:56:44] 0.110829 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:56:59] Energy consumed for RAM : 0.020875 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:56:59] Energy consumed for all GPUs : 0.071580 kWh. Total GPU Power : 48.660000000000004 W
[codecarbon INFO @ 19:56:59] Energy consumed for all CPUs : 0.018949 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:56:59] 0.111404 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:57:14] Energy consumed for RAM : 0.021070 kWh. RAM Power : 46.85093021392822 W
[codecarbon INFO @ 19:57:14] Energy consumed for all GPUs : 0.071784 kWh. Total GPU Power : 48.91600000000001 W
[codeca